<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Capstone-Project---The-Battle-of-the-Neighborhoods-(Week-2)" data-toc-modified-id="Capstone-Project---The-Battle-of-the-Neighborhoods-(Week-2)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Capstone Project - The Battle of the Neighborhoods (Week 2)</a></span><ul class="toc-item"><li><span><a href="#Introduction:-Business-Problem-" data-toc-modified-id="Introduction:-Business-Problem--1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction: Business Problem <a name="introduction"></a></a></span></li><li><span><a href="#Data-" data-toc-modified-id="Data--1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data <a name="data"></a></a></span></li><li><span><a href="#Methodology-" data-toc-modified-id="Methodology--1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Methodology <a name="methodology"></a></a></span></li><li><span><a href="#Analysis-" data-toc-modified-id="Analysis--1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Analysis <a name="analysis"></a></a></span><ul class="toc-item"><li><span><a href="#Installing-beautifulsoup4-and-geopy" data-toc-modified-id="Installing-beautifulsoup4-and-geopy-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Installing beautifulsoup4 and geopy</a></span></li><li><span><a href="#Importing-librairies" data-toc-modified-id="Importing-librairies-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Importing librairies</a></span></li><li><span><a href="#Scraping-data" data-toc-modified-id="Scraping-data-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Scraping data</a></span></li><li><span><a href="#Transforming-scraped-data-to-a-dataframe" data-toc-modified-id="Transforming-scraped-data-to-a-dataframe-1.4.4"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>Transforming scraped data to a dataframe</a></span></li><li><span><a href="#Cleaning-the-dataframe" data-toc-modified-id="Cleaning-the-dataframe-1.4.5"><span class="toc-item-num">1.4.5&nbsp;&nbsp;</span>Cleaning the dataframe</a></span></li><li><span><a href="#Exploring-Etobicoke" data-toc-modified-id="Exploring-Etobicoke-1.4.6"><span class="toc-item-num">1.4.6&nbsp;&nbsp;</span>Exploring Etobicoke</a></span></li><li><span><a href="#Using-Foursquare" data-toc-modified-id="Using-Foursquare-1.4.7"><span class="toc-item-num">1.4.7&nbsp;&nbsp;</span>Using Foursquare</a></span></li><li><span><a href="#Analyzing-each-neighborhood" data-toc-modified-id="Analyzing-each-neighborhood-1.4.8"><span class="toc-item-num">1.4.8&nbsp;&nbsp;</span>Analyzing each neighborhood</a></span></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-1.4.9"><span class="toc-item-num">1.4.9&nbsp;&nbsp;</span>Clustering</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li></ul></div>

# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant in **Etobicoke**. Specifically, this report will be targeted to stakeholders interested in opening a **restaurant** in **Etobicoke**, Canada.

Since there are lots of restaurants in York we will try to detect **locations that are not already crowded with restaurants**. 

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* Number of existing restaurants in the neighborhood (any type of restaurant)
* Some characteristics of neighborhoods

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to: 
* Extract/generate the required data by scraping it from the following website: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
* Transform and treat the scraped data
* Know the number of restaurants and their type and location in every neighborhood using **Foursquare API**

## Methodology <a name="methodology"></a>

First of all, we will scrap data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and we will transform it to a dataframe that we can use in our study. Next, we will convert addresses into their equivalent latitude and longitude values. Also, we will use the Foursquare API to explore neighborhoods in Etobicoke and we will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in Etobicoke and their emerging clusters.

## Analysis <a name="analysis"></a>

### Installing beautifulsoup4 and geopy

In [1]:
#!conda update -n base -c defaults conda
#!conda install -c anaconda beautifulsoup4 --yes
#!conda install -c conda-forge geopy --yes
#!conda install -c anaconda folium --yes

### Importing librairies

In [2]:
from bs4 import BeautifulSoup

import urllib.request

import pandas as pd

from geopy.geocoders import Nominatim

import folium

import requests

# Nupmy
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### Scraping data

In [3]:
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)

# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')

# find results within table
table = soup.find('table', attrs={'class': 'wikitable sortable'})
results = table.find_all('tr')

### Transforming scraped data to a dataframe

In [4]:
df = []
for tr in results:
    td = tr.find_all('td')
    row = [tr.text.rstrip("\n") for tr in td]
    df.append(row)

In [5]:
df1 = pd.DataFrame(df, columns =['Postal Code', 'Borough', 'Neighborhood'])

### Cleaning the dataframe

In [6]:
for i in range(len(df1)):
    if df1.loc[i,'Borough'] == 'Not assigned':
        df1.drop(i, inplace=True)

In [7]:
df1.reset_index(inplace=True)

In [8]:
for i in range(len(df1)):
    if df1.loc[i,'Neighborhood'] == 'Not assigned':
        df1.loc[i,'Neighborhood'] = df1.loc[i,'Borough'] 

In [9]:
df1.drop(['index'],axis=1 ,inplace=True)
df1.drop(0, inplace=True)

In [10]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [11]:
final_df = df1.merge(geo, how='outer')

In [12]:
final_df['Borough'].values

array(['North York', 'North York', 'Downtown Toronto', 'North York',
       'Downtown Toronto', 'Etobicoke', 'Scarborough', 'North York',
       'East York', 'Downtown Toronto', 'North York', 'Etobicoke',
       'Scarborough', 'North York', 'East York', 'Downtown Toronto',
       'York', 'Etobicoke', 'Scarborough', 'East Toronto',
       'Downtown Toronto', 'York', 'Scarborough', 'East York',
       'Downtown Toronto', 'Downtown Toronto', 'Scarborough',
       'North York', 'North York', 'East York', 'Downtown Toronto',
       'West Toronto', 'Scarborough', 'North York', 'North York',
       'East York', 'Downtown Toronto', 'West Toronto', 'Scarborough',
       'North York', 'North York', 'East Toronto', 'Downtown Toronto',
       'West Toronto', 'Scarborough', 'North York', 'North York',
       'East Toronto', 'Downtown Toronto', 'North York', 'North York',
       'Scarborough', 'North York', 'North York', 'East Toronto',
       'North York', 'York', 'North York', 'Scarborough', 'Nort

### Exploring Etobicoke

In [13]:
etobicoke_data = final_df[final_df['Borough'] == 'Etobicoke'].reset_index(drop=True)

In [14]:
etobicoke_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
5,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
6,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
7,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
8,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [15]:
address = 'Etobicoke'

geolocator = Nominatim(user_agent="ek_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [16]:
# create map of York using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

### Using Foursquare

In [17]:
CLIENT_ID = 'A3I4UY5CIRGXQ132ZZN5NELPGY1JQ4Q5VQUWPULTPLBJWT3B' # your Foursquare ID
CLIENT_SECRET = 'E21H5H2UXBSAU22FHOBFRKMXAPIFXQH1AUS5DFVLLOYQ3EGX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Alderwood, Long Branch
Northwest, West Humber - Clairville
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West


### Analyzing each neighborhood

In [20]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
toronto_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,Coffee Shop,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Social Club,Supplement Shop,Tanning Salon,Wings Joint,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"West Deane Park, Princess Gardens, Martin Grov..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."


In [21]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,...,Pizza Place,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Social Club,Supplement Shop,Tanning Salon,Wings Joint
0,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.285714,0.142857,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.000000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.000000,0.0,0.142857,0.000000,0.00,0.142857,0.000000,...,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Kingsview Village, St. Phillips, Martin Grove ...",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,...,0.250000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.000000
3,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.066667,0.000000,0.0,0.000000,0.066667,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.066667,0.066667,0.066667,0.066667,0.066667
4,"New Toronto, Mimico South, Humber Bay Shores",0.071429,0.071429,0.000000,0.0,0.000000,0.000000,0.00,0.142857,0.000000,...,0.071429,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Northwest, West Humber - Clairville",0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.00,0.000000,0.000000,...,0.111111,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.000000
8,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,"West Deane Park, Princess Gardens, Martin Grov...",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3        Pharmacy  0.14
4     Coffee Shop  0.14


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
          venue  freq
0  Liquor Store  0.14
1    Beer Store  0.14
2          Café  0.14
3     Pet Store  0.14
4   Coffee Shop  0.14


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0          Pizza Place  0.25
1       Sandwich Place  0.25
2             Bus Line  0.25
3    Mobile Phone Shop  0.25
4  American Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
             venue  freq
0              Gym  0.07
1   Discount Store  0.07
2    Tanning Salon  0.07
3  Supplement Shop  0.07
4      Social Club  0.07


----New Toronto, Mimico South, Humber Bay Shores----
                  venue  freq
0                  Café  

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Gym,Pub,Coffee Shop,Pharmacy,Sandwich Place,Baseball Field,Beer Store,Burger Joint,Bar
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pharmacy,Coffee Shop,Beer Store,Liquor Store,Café,Pizza Place,Pet Store,Fast Food Restaurant,Drugstore,Discount Store
2,"Kingsview Village, St. Phillips, Martin Grove ...",Mobile Phone Shop,Sandwich Place,Bus Line,Pizza Place,Wings Joint,Filipino Restaurant,Fast Food Restaurant,Drugstore,Discount Store,Deli / Bodega
3,"Mimico NW, The Queensway West, South of Bloor,...",Wings Joint,Kids Store,Bakery,Burger Joint,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Grocery Store,Tanning Salon
4,"New Toronto, Mimico South, Humber Bay Shores",Café,Gym,Pharmacy,Bakery,Coffee Shop,Fast Food Restaurant,Fried Chicken Joint,Liquor Store,Mexican Restaurant,Pet Store


### Clustering

In [25]:
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 3, 2, 4, 1])

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etobicoke_merged = etobicoke_data

etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etobicoke_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,1.0,Filipino Restaurant,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Deli / Bodega,Convenience Store,Chinese Restaurant
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,2.0,Pharmacy,Coffee Shop,Beer Store,Liquor Store,Café,Pizza Place,Pet Store,Fast Food Restaurant,Drugstore,Discount Store
3,M9P,Etobicoke,Westmount,43.696319,-79.532242,2.0,Coffee Shop,Sandwich Place,Intersection,Discount Store,Pizza Place,Middle Eastern Restaurant,Chinese Restaurant,Wings Joint,Fast Food Restaurant,Drugstore
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,2.0,Mobile Phone Shop,Sandwich Place,Bus Line,Pizza Place,Wings Joint,Filipino Restaurant,Fast Food Restaurant,Drugstore,Discount Store,Deli / Bodega


## Results

As you can see we reached the objects that we talked about in the introduction.

## Conclusion

This project was a very interesting application of data science. It helped me learn many new notions and practice what I learnt in the IBM courses.